In [8]:
import datetime
import pandas as pd 
import numpy as np
import requests
from sklearn.externals import joblib
import numpy as np
import pandas as pd
from dayu import settings
from dayu.hooks.mysql_hook import MySqlHook
import logging
import datetime
from logging.handlers import TimedRotatingFileHandler
from logging.handlers import RotatingFileHandler
from sklearn.externals import joblib
from collections import OrderedDict
import sys
from dayu.hooks.oss_hook import OSSHook

from dayu.hooks.hive_server_hook import HiveServerHook
from dayu.hooks.hive_cli_hook import HiveCliHook

import time
import requests
import warnings
warnings.filterwarnings('ignore')

import csv, math
#import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from dayu import settings
import os 


def split_table_name(datain):
    new_cols = []
    for column in datain.columns:
        if(len(column.split('.'))<2):
            new_cols.append(column)
        else:
            new_cols.append(column.split('.')[1])

    datain.columns = new_cols
    return datain

def read_from_hive2(output_file_name,insql,dtype):
    filename = output_file_name
    filepath = curr_dir+filename
    hive = HiveServerHook("warehouse_hive")
    hive.to_csv(insql,filepath , delimiter=',',lineterminator='\n', output_header=True)
    outdata = pd.read_csv(filepath, header=0,dtype=dtype)
    # 去除列名中带有的表名
    outdata = split_table_name(outdata)
    return outdata

"""
    自定义函数
"""
def date_time_sub(startTime,endTime,date_format):
    try:
        startTime= datetime.datetime.strptime(startTime,date_format)
        endTime= datetime.datetime.strptime(endTime,date_format)
        return (endTime - startTime).days
    except:
        pass

def get_hive_data(hql):
    with data_source.connect_hive(key) as conn:
        # key 是数据源的key, 必须是分配到任务所在项目的数据源才能连上
        # 使用with生成一个cursor, 用来执行语句
        with conn.cursor() as cursor:
            cursor.execute(hql, parameters=None, verbose=False)
            print(cursor.description)
            column_list = []
            for col in cursor.description:
                if len(col[0].split('.')) == 2:
                    column_list.append(col[0].split('.')[1])
                else:
                    column_list.append(col[0])

            data_list = cursor.fetchall() # 获取所有执行结果
            data_list = [d for d in data_list]

    data_df = pd.DataFrame(data_list,columns=column_list)
    return data_df
## 数据结果评估
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

class Logger:       
    def __init__(self, logName, logFile):
        self._logger = logging.getLogger(logName)
        handler = logging.FileHandler(logFile)
        formatter = logging.Formatter('%(asctime)s ********* %(message)s')
        handler.setFormatter(formatter)
        self._logger.addHandler(handler)
        self._logger.setLevel(logging.INFO)

    def log(self, msg):
        if self._logger is not None:
            self._logger.info(msg)

curr_date = str(datetime.datetime.now())[0:10]
pd.set_option('display.max_columns', 500)
curr_dir = settings.TEMP_FILE_DIR+'/'
hive_cli = HiveCliHook("warehouse_hive")
curr_dir = '/home/souche/qiongjiu/hgc/'

logger = Logger('model_service','./log/accurate_valuation_cyp_run_log.log')
logger.log("程序启动.............")


[2021-02-03 16:19:41,836] {<ipython-input-8-1cbf1b0aa466>:114} INFO - 程序启动.............


In [3]:
hql = """
select * from db_data.enterprise2_car_parameter_onehot
where ds = date_sub('2020-12-01',1) 
"""
dtype={'city_code':str}
car = read_from_hive2('model',hql,dtype)


[2021-02-03 16:16:09,908] {hiveserver2:138} INFO - Using database default as default
[2021-02-03 16:16:10,048] {hive_server_hook:112} INFO - Running query: 
select * from db_data.enterprise2_car_parameter_onehot
where ds = date_sub('2020-12-01',1) 

[2021-02-03 16:16:17,111] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2021-02-03 16:16:23,831] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2021-02-03 16:16:30,355] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2021-02-03 16:16:36,923] {hive_server_hook:162} INFO - Written 40000 rows so far.
[2021-02-03 16:16:43,062] {hive_server_hook:162} INFO - Written 50000 rows so far.
[2021-02-03 16:16:49,699] {hive_server_hook:162} INFO - Written 60000 rows so far.
[2021-02-03 16:16:53,261] {hive_server_hook:162} INFO - Written 65632 rows so far.
[2021-02-03 16:16:53,314] {hiveserver2:265} INFO - Closing active operation
[2021-02-03 16:16:53,329] {hive_server_hook:163} INFO - Done. Loaded a total of 65632 rows.


In [4]:
car = car.drop(['driving_mode', 'gear_box_type', 'country_id',
                'import_type', 'intake_type', 'fuel_form',
                'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7', 'year_8',
                'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 'year_14', 'year_15',
                'year_16', 'car_body', 'rate', 'rate_count', 'series_level', 'level0',
                'level1', 'level2','level3','level4','level5','level6','level7','level8'], axis=1)

In [13]:
# 导出车型参数及独热编码
car_all = pd.read_csv(curr_dir+"2021-02-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car = car_all[['model_code', 'model_year', 'new_car_price', 'wheel_base', 'length', 'height', 'width', 'max_torque', 'max_power', 'engine_volume_l', 'cylinder_number', 'level', 'seat_number_top', 'quality_mile', 'quality_year', 'driving_mode0', 'driving_mode1', 'driving_mode2', 'driving_mode3', 'driving_mode4', 'driving_mode5', 'driving_mode6', 'driving_mode7', 'driving_mode8', 'driving_mode9', 'gear_box_type0', 'gear_box_type1', 'gear_box_type2', 'gear_box_type3', 'gear_box_type4', 'gear_box_type5', 'gear_box_type6', 'gear_box_type7', 'gear_box_type8', 'gear_box_type9', 'gear_box_type10', 'country_id0', 'country_id1', 'country_id2', 'country_id3', 'country_id4', 'country_id5', 'country_id6', 'country_id7', 'country_id8', 'country_id9', 'country_id10', 'country_id11', 'country_id12', 'country_id13', 'country_id14', 'import_type0', 'import_type1', 'intake_type0', 'intake_type1', 'intake_type2', 'intake_type3', 'intake_type4', 'intake_type5', 'intake_type6', 'intake_type7', 'fuel_form0', 'fuel_form1', 'fuel_form2', 'fuel_form3', 'fuel_form4', 'fuel_form5', 'fuel_form6', 'fuel_form7', 'fuel_form8', 'car_body0', 'car_body1', 'car_body2', 'car_body3', 'car_body4', 'car_body5', 'car_body6', 'car_body7', 'car_body8', 'car_body9', 'car_body10', 'car_body11', 'series_level0', 'series_level1', 'series_level2', 'series_level3', 'series_level4', 'series_level5', 'series_level6', 'series_level7', 'series_level8', 'series_level9', 'series_level10', 'series_level11', 'series_level12', 'series_level13', 'series_level14', 'series_level15', 'series_level16', 'series_level17', 'series_level18', 'series_level19', 'series_level20', 'series_level21', 'series_level22', 'series_level23', 'series_level24', 'series_level25', 'series_level26', 'series_level27', 'series_level28', 'series_level29', 'series_level30', 'series_level31', 'series_level32', 'series_level33', 'series_level34', 'series_level35', 'series_level36', 'series_level37', 'series_level38', 'series_level39', 'series_level40', 'series_level41', 'series_level42', 'series_level43', 'series_level44', 'series_level45', 'series_level46', 'series_level47', 'series_level48', 'series_level49', 'series_level50', 'series_level51', 'series_level52', 'series_level53', 'series_level54', 'series_level55', 'series_level56', 'series_level57', 'series_level58', 'series_level59', 'series_level60', 'series_level61', 'series_level62']]

A=["Mini Bus",'BUS']
car.loc[car.level.isin(A), 'level'] = "A"
ph = curr_dir+"enterprise2_car_parameter_onehot.csv"
car.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/全网数据-优化模型/"+curr_date+"/enterprise2_car_parameter_onehot.csv", ph)
print("enterprise2_car_parameter_onehot 文件ok........")

## 导出TF-IDF权重的词向量
vec = pd.read_csv(curr_dir+"2021-02-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
ph = curr_dir+"enterprise2_model_name_word2vec.csv"
vec.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/全网数据-优化模型/"+curr_date+"/enterprise2_model_name_word2vec.csv", ph)
print("enterprise2_model_name_word2vec 文件ok........")
## 导出保值率和汽车之家评论数据
hql = """
select model_code,series_code,brand_code from db_data.ods_car_model_model 
where ds = date_sub('2021-02-02',1) 
"""
dtype={'city_code':str}
model_info = read_from_hive2('model',hql,dtype)

car_all = pd.merge(car_all, model_info,on='model_code',how='left')

keep_value = car_all[['model_code', 'year_1', 'year_2', 'year_3', 'year_4', 'year_5', 'year_6', 'year_7', 'year_8',
                         'year_9', 'year_10', 'year_11', 'year_12', 'year_13', 'year_14', 'year_15',
                         'year_16', 'rate', 'rate_count','series_code','brand_code']]

ph = curr_dir+"enterprise2_keep_value_and_rate.csv"
keep_value.to_csv(ph)
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/全网数据-优化模型/"+curr_date+"/enterprise2_keep_value_and_rate.csv", ph)
print("enterprise2_keep_value_and_rate 文件ok........")



[2021-02-03 16:26:40,781] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/全网数据-优化模型/2021-02-03/enterprise2_car_parameter_onehot.csv .
enterprise2_car_parameter_onehot 文件ok........
[2021-02-03 16:26:56,393] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/全网数据-优化模型/2021-02-03/enterprise2_model_name_word2vec.csv .
enterprise2_model_name_word2vec 文件ok........
[2021-02-03 16:26:56,413] {hiveserver2:138} INFO - Using database default as default
[2021-02-03 16:26:56,528] {hive_server_hook:112} INFO - Running query: 
select model_code,series_code,brand_code from db_data.ods_car_model_model 
where ds = date_sub('2021-02-02',1) 

[2021-02-03 16:26:56,903] {hive_server_hook:162} INFO - Written 10000 rows so far.
[2021-02-03 16:26:57,076] {hive_server_hook:162} INFO - Written 20000 rows so far.
[2021-02-03 16:26:57,248] {hive_server_hook:162} INFO - Written 30000 rows so far.
[2021-02-03 16:26:57,421] {hive_server_hook:162} INFO - Written 40000 rows so